In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.signal import find_peaks

In [2]:
class parE:
    def _init_(self):
        self.param = [0]*34
        
class PopAverage:
    k = 0
    n = 0
    Km = 0
    L = 0
    period = 0
    amp = 0
    tau = 0
    w = 0
    f = 0
    def reset(self):
        self.k = 0
        self.n = 0
        self.Km = 0
        self.L = 0
        self.period = 0
        self.amp = 0
        self.tau = 0
        self.w = 0
        self.f = 0
        
    def normalize(self,N):
        f = self.f
        self.k /= f
        self.n /= f
        self.Km /= f
        self.L /= f
        self.period /= f
        self.amp /= f
        self.tau /= f
        self.w /= f
        self.f /= N
        

In [3]:
class GeneNetwork:
    k = []
    n = []
    KM = []
    L = 0
    kd = []
    x = []
    LD=0
    tau = 0
    period = 0
    amp = 0
    v = 0.02
    dawn = 10000
    dusk = 30000
    w = 0
    parent = 0
    mut = 0
    LTag = 0
    LMut = 0
    LOsc = 0
    LFit = 0
    r = []
    rT = 0
    V = [[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1],
         [0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0],
         [-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1]]
    time = 0
    y=[]
    t=[0]
    
    def ic(self,x0,k0,L0,n0,KM0,Kd0):
        self.x = x0
        self.k = k0
        self.L = L0
        self.n = n0
        self.KM = KM0
        self.kd = Kd0
        self.y = [[i] for i in self.x]
        
    def updateR(self):
        x = self.x
        k = self.k
        L = self.L
        n = self.n
        KM = self.KM
        kd = self.kd
        
        m = self.time%24
        if m>=12 or not self.LD:
            l = 0
        else:
            l = L
        
        self.r[0] = k[0]
        self.r[1] = k[1]*x[5] 
        self.r[2] = k[2]*x[6]
        self.r[3] = k[3]/(1 + KM[0]*x[5])
        self.r[4] = k[4]*x[11]
        self.r[5] = k[5]*x[0] 
        self.r[6] = k[6]*(x[0] + x[10] + KM[1] -  sqrt(pow((x[0] + x[10] + KM[1]),2) - 4*x[0]*x[10]))/2 
        self.r[7] = k[7]*x[1] 
        self.r[8] = k[8]*x[2]*x[3]
        self.r[9] = k[9]*x[4] 
        self.r[10] = k[10]*x[6] 
        self.r[11] = k[11]*x[5]*(pow(x[11],n[0])/(pow(x[11],n[0]) + pow(KM[2],n[0]))) 
        self.r[12] = k[12]*x[5]
        self.r[13] = k[13]*x[6]*(pow(x[14],n[1])/(pow(x[14],n[1]) + pow(KM[3],n[1]))) 
        self.r[14] = (k[14]*pow((KM[5]*x[5]),n[2]))/(pow((KM[4]*KM[5]),n[2]) + pow((KM[5]*x[5]),n[2]) + pow((KM[4]*x[6]),n[2])) 
        self.r[15] = (k[15]*pow((KM[4]*x[6]),n[2]))/(pow((KM[4]*KM[5]),n[2]) + pow((KM[5]*x[5]),n[2]) + pow((KM[4]*x[6]),n[2])) 
        self.r[16] = k[16]*x[7]
        self.r[17] = k[17]*x[8] 
        self.r[18] = k[18]*x[9] 
        self.r[19] = k[19]*x[10] 
        self.r[20] = k[20]*(pow(x[6],n[3])/(pow(KM[6],n[3]) + pow(x[6],n[3]))) 
        self.r[21] = k[21]*x[12]
        self.r[22] = k[22]*x[13]
        self.r[23] = l*x[6] 
        self.r[24] = kd[0]*x[0] 
        self.r[25] = kd[1]*x[1] 
        self.r[26] = kd[2]*x[2] 
        self.r[27] = kd[3]*x[3] 
        self.r[28] = kd[4]*x[4] 
        self.r[29] = kd[5]*x[5]
        self.r[30] = kd[6]*x[6] 
        self.r[31] = kd[7]*x[7] 
        self.r[32] = kd[8]*x[8]
        self.r[33] = kd[9]*x[9] 
        self.r[34] = kd[10]*x[10] 
        self.r[35] = kd[11]*x[11] 
        self.r[36] = kd[12]*x[12] 
        self.r[37] = kd[13]*x[13] 
        self.r[38] = kd[14]*x[14]
        
        self.rT = sum(self.r)
    
    def updateTime(self):
        rand1 = np.random.rand()
        
        updateR()
        re_time = -np.log(1-rand1)/rT
        self.time += re_time
        self.t.append(self.time)
    
    def updateSystem(self):
        r = self.r
        rT = self.rT
        rand2 = np.random.rand()
        rR = sum(rand2>(np.cumsum(r)/rT))
        self.x += self.V[rR]
        x=[[i] for i in self.x]
        self.y = np.append(self.y,x,axis=1)
    
    def ODE(self,tm,y):
        def f_m(P_1,P_2,K_m):
            return (P_1 + P_2 + K_m -  np.sqrt(pow((P_1 + P_2 + K_m),2) - 4*P_1*P_2))/2
            
        wc1 = y[0]
        wc2 = y[1]
        WC1 = y[2]
        WC2 = y[3]
        WCCc = y[4]
        WCCn = y[5]
        laWCC = y[6]
        frq = y[7]
        FRQ = y[8]
        FFC = y[9]
        FFCp = y[10]
        FFCn = y[11]
        vvd = y[12]
        VVDc = y[13]
        VVDn = y[14]
        
        k=self.k
        L=self.L
        n=self.n
        K=self.KM
        kd=self.kd
        
        #Tiempo y periodos LD
        m=tm%24
        if m>=12 or not self.LD:
            l = 0
        else:
            l = L
    
        #Ecuaciones        
        dwc1 = k[0] + k[1]*WCCn + k[2]*laWCC - kd[0]*wc1
        dwc2 = k[3]/(1 + K[0]*WCCn) + k[4]*FFCn - kd[1]*wc2
        dWC1 = k[5]*wc1 + k[6]*f_m(wc1,FFCp,K[1]) - k[8]*WC1*WC2 - kd[2]*WC1
        dWC2 = k[7]*wc2 - kd[3]*WC2 - k[8]*WC1*WC2
        dWCCc = k[8]*WC1*WC2 - kd[4]*WCCc -  k[9]*WCCc
        dWCCn = k[9]*WCCc + k[10]*laWCC - kd[5]*WCCn - l*WCCn - k[12]*WCCn - k[11]*WCCn*(pow(FFCn,n[0])/(pow(K[2],n[0]) + pow(FFCn,n[0])))
        dlaWCC = L*WCCn + k[12]*WCCn - kd[6]*laWCC - k[10]*laWCC - k[13]*laWCC*(pow(VVDn,n[1])/(pow(K[3],n[1]) + pow(VVDn,n[1])))
        dfrq =  (k[14]*pow((K[5]*WCCn),n[2]) + k[15]*pow((K[4]*laWCC),n[2]))/(pow((K[4]*K[5]),n[2]) + pow((K[5]*WCCn),n[2]) + pow((K[4]*laWCC),n[2])) - kd[7]*frq
        dFRQ =  k[16]*frq - kd[8]*FRQ - k[17]*FRQ
        dFFC = k[17]*FRQ - kd[9]*FFC - k[18]*FFC
        dFFCp = k[18]*FFC - kd[10]*FFCp - k[19]*FFCp
        dFFCn = k[19]*FFCp - kd[11]*FFCn
        dvvd = (k[20]*pow(laWCC,n[3]))/(pow(K[6],n[3]) + pow(laWCC,n[3])) - kd[12]*vvd
        dVVDc = k[21]*vvd - kd[13]*VVDc -  k[22]*VVDc
        dVVDn = k[22]*VVDc - kd[14]*VVDn

        return [dwc1, dwc2, dWC1, dWC2, dWCCc, dWCCn, dlaWCC, dfrq, dFRQ, dFFC, dFFCp, dFFCn, dvvd, dVVDc, dVVDn]
    
    #def calculateFit(self):
    #    t = self.t
    #    y = self.y[5]
    #    L
    #    if int(self.time)%24:
    #        Popt = 22.5
    #    else:
    #        Popt = 24
    #        
    #    dis = int(len(t)/24)
    #    peaks, _ = find_peaks(y, distance = dis)
    #    self.period = np.diff(t[peaks[-2:]])
    #    self.amp = (y[peaks[-2]] + y[peaks[-1]] - min(y[peaks[-2]:peaks[-1]]) - min(y[peaks[-1]:]))/2
    #    
    #    for i,j in enumerate(t):
    #        if j>self.period:
    #            d_ind = i
    #            break
    #    for i,j in enumerate(t):
    #        if j>(t[-1]-self.period):
    #            d_ind += len(t)-i
    #            break
    #    d_ind /= 2
    #    d_ind = int(d_ind)
    #    me = np.mean(y)
    #    acf = np.mean((y[0:-d_ind]-me)*(y[d_ind:]-me))
    #    s = np.mean((y-me)*(y-me))
    #    d = np.abs(self.period - Popt)
    #    if acf!=0 and s!=0 and acf!=s:
    #        self.tau = np.abs(1/np.log(acf/s))*self.period
    #        self.w = np.exp(-d/self.tau)
    #    elif acf == s:
    #        self.tau = np.inf
    #        self.w = 1
    #    else:
    #        self.tau = 0
    #        self.w = 0
    
    def calculateFit(self):
        WCC = self.x[5]
        time = int(self.time)
        v = self.v
        amp = (self.dusk-self.dawn)/2
        yd = self.dawn + amp
        WCCopt = yd + amp*(np.cos(2*np.pi*(time+12)/self.period))
        self.w =  1/(1+((WCC-WCCopt)**2/(v*WCCopt)))
    
    def mutate(self,M,par_E):
        
        randl = np.random.rand(35)
        qm = pow(M,(1-(2*randl)))
        qm *= par_E.param
        qm = np.where(qm>0,qm,1)
        
        self.k *= qm[0:23]
        self.n *= qm[23:27]
        self.KM *= qm[27:34]
        self.L *= qm[34]
        
        self.mut = 1
        self.LMut += 1

In [4]:
def selectionTournament(myPop, N, st, u, M, par_E):
    mutscenarios = [[1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                    [0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
                    [0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0],
                    [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,1]]
    

    indexT = [np.random.randint(N)]
    maxT = indexT[0]
    j = 1
    while j<st:
        rnum = np.random.randint(N)
        if not rnum in indexT:
            indexT.append(rnum)
            if myPop[maxT].w < myPop[indexT[j]].w:
                maxT = indexT[j]
            j+=1
    j = np.random.randint(N)
    myPop[j] = myPop[maxT]
    myPop[j].parent = maxT
    myPop[j].y = [[i] for i in myPop[j].x]
    myPop[j].t = [myPop[j].time]
    if np.random.rand() <= u:
        par_E.param = mutscenarios[np.random.randint(8)]
        myPop[j].mutate(M,par_E)

In [5]:
TMAX = 2
rRE = [0.3,0.4,0.5,0.6]
GMAX = 10000
Cp = 24
period = 24
x0 = [12700, 15819, 496, 332515, 8851, 1338, 1, 99, 491, 25, 103, 248, 0, 12, 1607]
k0 = [2975000, 1.2, 90, 4000000, 0.03, 0.226, 2.4, 2, 1.888e-07, 0.3, 0.001, 50, 0.001, 20, 18250000, 800000000, 5.4, 0.15, 2, 0.05, 2000000000, 0.68, 0.3]
L0 = 0.1
n0 = [2,2,3,3]
Km0 = [1.2e-8, 750000, 125000, 12500000, 5000000, 450000, 50000]
kd0 = [2.4, 2.5, 0.135, 0.085, 0.05, 0.05, 6, 1, 0.69, 0.34, 0.34, 0.1, 6.2, 0.24, 0.24]
par_E = parE()
numrep = 3

In [6]:
#N = input("Number of cells")
#st = input("Selection strength")
#u = input("Mutation rate")
#M = input("Mutation factor")
#alg = input("Type of algorithm to simulate with (0 = Deterministic, 1 = Gillespie)")
#LD = input("Type of environment condition to simulate with (0 = LD 12:12, 1 = DD)")

Number of cells 50
Selection strength 25
Mutation rate 0.03
Mutation factor 1.1
Type of algorithm to simulate with (0 = Deterministic, 1 = Gillespie) 0
Type of environment condition to simulate with (0 = LD 12:12, 1 = DD) 0


In [7]:
N = 500 #int(N)
st = 250 #float(st)
u = 0.03 #float(u)
M = 1.1 #float(M)
alg = 0 #int(alg)
LD = 0 #int(LD)


#Réplicas
for iS in range(0,numrep):
    myPop = [GeneNetwork() for i in range(0,N)]
    myPopT = PopAverage()
    resultAGS = []
    resultLxC = []
    resultI = []
    tGen = 0
    tCycle = 0
    iG = 0

    for i in range(0,N):
        myPop[i].ic(x0,k0,L0,n0,Km0,kd0)
        myPop[i].LTag = i
        myPop[i].LD = LD
        myPop[i].period = period
    #Generaciones
    while iG < TMAX*GMAX:
        #linajes
        iR = int((iG%24)//6)
        rLE = int((N*rRE[iR])/TMAX)
        tWait = -np.log(1 - np.random.rand())/rLE
        iG += tWait

        for i in range(0,N):
            if alg:
                while myPop[i].time < iG:
                    myPop[i].updateTime()
                    myPop[i].updateSystem()
            else:
                sol = solve_ivp(myPop[i].ODE,[myPop[i].time,iG],myPop[i].x)
                myPop[i].t = np.append(myPop[i].t,sol.t)
                myPop[i].y = np.append(myPop[i].y,sol.y.round(4),axis=1)
                myPop[i].time = myPop[i].t[-1]
                myPop[i].x = myPop[i].y[:,-1]
            myPop[i].calculateFit()
            myPop[i].LFit += (myPop[i].w*tWait)
            tCycle += tWait
            
            myPopT.k += np.array(np.array(myPop[i].k)*tWait, dtype = float)
            myPopT.Km += np.array(np.array(myPop[i].KM)*tWait, dtype = float)
            myPopT.L += myPop[i].L*tWait
            myPopT.n += np.array(np.array(myPop[i].n)*tWait, dtype = float)
            myPopT.w += myPop[i].w*tWait
            myPopT.period += myPop[i].period*tWait
            myPopT.amp += myPop[i].amp*tWait
            myPopT.tau += myPop[i].tau*tWait
            myPopT.f += tWait
            #
            if int(iG/TMAX)>tGen:
                resultI.append([tGen,i,LD,myPop[i].k,myPop[i].L,myPop[i].KM,myPop[i].n,
                                #myPop[i].period,myPop[i].amp,myPop[i].tau,
                                myPop[i].w,myPop[i].y[5],myPop[i].parent])
                if tGen%Cp==0:
                    resultLxC.append([tGen,LD,myPop[i].LTag,myPop[i].LMut,myPop[i].LFit/tCycle,myPop[i].k,
                                      myPop[i].L,myPop[i].KM,myPop[i].n])
                    myPop[i].LFit = 0
                    myPop[i].LMut = 0
                    myPop[i].LTag = i
            
        if int(iG/TMAX)> tGen:
            tGen=iG//TMAX
            myPopT.normalize(N)
            resultAGS.append([tGen,LD,myPopT.k,myPopT.L,myPopT.Km,myPopT.n,
                              #myPopT.period,myPopT.amp,myPopT.tau,
                              myPopT.w])
            myPopT.reset()
        selectionTournament(myPop, N, st, u, M, par_E)
        
    resultI = pd.DataFrame(resultI,columns=["Generation","Cell tag","Environment (LD=0 or DD=1)",
                                            "Expresion and repression","Constant of light rate",
                                            "Disociation Constants", "Hill exponents",
                                            #"Period","Amplitude","Tau",
                                            "Fitness","WCCn","Parent tag"])
    
    resultLxC = pd.DataFrame(resultLxC,columns=["Generation","Environment (LD=0 or DD=1)","LTag","LMut","LFit",
                                            "Expresion and repression","Constant of light rate",
                                            "Disociation Constants", "Hill exponents"])
    
    resultAGS = pd.DataFrame(resultAGS,columns=["Generation","Environment (LD=0 or DD=1)",
                                                "Expresion and repression constants","Constant of light rate",
                                                "Disociation constants", "Hill exponents",
                                                #"Period","Amplitude","Tau",
                                                "Fitness"])
    
    resultI.to_csv("Result\Ind_inf_u{}_M{}_st{}_N{}_LD{}_alg{}_{}.csv".format(u,M,st,N,LD,alg,iS))
    resultLxC.to_csv("Result\LineagexCycle_u{}_M{}_st{}_N{}_LD{}_alg{}_{}.csv".format(u,M,st,N,LD,alg,iS))
    resultAGS.to_csv("Result\AveGenSub_u{}_M{}_st{}_N{}_LD{}_alg{}_{}.csv".format(u,M,st,N,LD,alg,iS))
    

0.0